In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [20]:
df = pd.read_csv('beer_reviews/beer_reviews/beer_reviews.csv', header = 0)
df = df.drop('brewery_name', axis=1)
df = df.drop('review_profilename', axis=1)
df = df.drop('beer_style', axis=1)
df = df.drop('beer_name', axis=1)
df = df[np.isfinite(df['beer_abv'])]

In [29]:
df.info()
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
0,10325,1234817823,1.5,2.0,2.5,1.5,1.5,5.0,47986
1,10325,1235915097,3.0,2.5,3.0,3.0,3.0,6.2,48213
2,10325,1235916604,3.0,2.5,3.0,3.0,3.0,6.5,48215
3,10325,1234725145,3.0,3.0,3.5,2.5,3.0,5.0,47969
4,1075,1293735206,4.0,4.5,4.0,4.0,4.5,7.7,64883
5,1075,1325524659,3.0,3.5,3.5,3.0,3.5,4.7,52159
6,1075,1318991115,3.5,3.5,3.5,4.0,4.0,4.7,52159
7,1075,1306276018,3.0,2.5,3.5,2.0,3.5,4.7,52159
8,1075,1290454503,4.0,3.0,3.5,3.5,4.0,4.7,52159
9,1075,1285632924,4.5,3.5,5.0,4.0,4.0,4.7,52159


In [30]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(df)